In [1]:
import tensorflow as tf
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

In [2]:
X = pd.read_csv(r'../data/X_expr.csv').drop(['Unnamed: 0', 'seqLibID'], axis=1).values
y = pd.read_csv(r'../data/y_cog.csv').drop(['Unnamed: 0', 'seqLibID'], axis=1).values

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

train_ds = tf.data.Dataset.from_tensor_slices(
    (X_train, y_train)
).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

NameError: name 'train_test_split' is not defined

In [4]:
class BaseDNN(tf.keras.Model):
    def __init__(self, node_counts, dropout_ratios=None, 
                 activation='relu', optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True)):
        super(BaseDNN, self).__init__()
        self.node_counts = node_counts
        self.dropout_ratios = [0 for _ in range(len(self.node_counts))] if dropout_ratios is None else dropout_ratios
        
        self.activation = activation
        self.optimizer = optimizer
        self.loss = loss

        # self.layer_list = []
        for i in range(len(self.node_counts)):
            n_nodes = self.node_counts[i]
            dropout_ratio = self.dropout_ratios[i]
            setattr(self, f'dense_{i}', layers.Dense(n_nodes, activation=self.activation))
            # setattr(self, f'dropout_{i}', layers.Dropout(dropout_ratio))

    def call(self, input_tensor):
        x = input_tensor
        for i in range(len(self.node_counts)):
            x = getattr(self, f'dense_{i}')(x)
            # x = getattr(self, f'dropout_{i}')(x)
        return x


class ConvolutionalLayers(tf.keras.Model):
    def __init__(self, filters, kernel_sizes, activation='relu', strides=None, paddings=None,
                 optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True)):
        super(ConvolutionalLayers, self).__init__()
        self.filters = filters
        self.kernels_sizes = kernel_sizes
        self.activation = activation

        if strides is None:
            self.strides = []
            for _ in range(len(filters)):
                self.strides.append((1, 1))
        else:
            self.strides = strides

        if paddings is None:
            self.paddings = []
            for _ in range(len(filters)):
                self.paddings.append("valid")
        else:
            self.paddings = paddings

        self.optimizer = optimizer
        self.loss = loss

        for i in range(len(self.filters)):
            n_filters = self.filters[i]
            kernel_size = self.kernels_sizes[i]
            stride = self.strides[i]
            padding = self.paddings[i]

            setattr(self, f'conv2d_{i}', layers.Conv2D(n_filters, kernel_size, strides=stride, padding=padding, activation=self.activation))

    def call(self, input_tensor):
        x = input_tensor
        for i in range(len(self.filters)):
            x = getattr(self, f'conv2d_{i}')(x)
        return x

In [5]:
class BaseDNN1(tf.keras.Model):
    def __init__(self):
        super(BaseDNN1, self).__init__()
        self.conv2d = layers.Conv2D(32, 3, activation='relu')
        self.flatten = layers.Flatten()
        self.dense_1 = layers.Dense(128, activation='relu')
        self.dense_2 = layers.Dense(10, activation='relu')

    def call(self, input_tensor):
        x = input_tensor
        x = self.conv2d(x)
        x = self.flatten(x)
        x = self.dense_1(x)
        x = self.dense_2(x)
        return x


class ConvolutionalLayers1(tf.keras.Model):
    def __init__(self, filters, kernel_sizes, activation='relu', strides=None, paddings=None,
                 optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True)):
        super(ConvolutionalLayers, self).__init__()
        self.filters = filters
        self.kernels_sizes = kernel_sizes
        self.activation = activation

        if strides is None:
            self.strides = []
            for _ in range(len(filters)):
                self.strides.append((1, 1))
        else:
            self.strides = strides

        if paddings is None:
            self.paddings = []
            for _ in range(len(filters)):
                self.paddings.append("valid")
        else:
            self.paddings = paddings

        self.optimizer = optimizer
        self.loss = loss

        for i in range(len(self.filters)):
            n_filters = self.filters[i]
            kernel_size = self.kernels_sizes[i]
            stride = self.strides[i]
            padding = self.paddings[i]

            setattr(self, f'conv2d_{i}', layers.Conv2D(n_filters, kernel_size, strides=stride, padding=padding, activation=self.activation))

    def call(self, input_tensor):
        x = input_tensor
        for i in range(len(self.filters)):
            x = getattr(self, f'conv2d_{i}')(x)
        return x

In [6]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

optimizer = tf.keras.optimizers.Adam()
loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

@tf.function
def train_step(model, data, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(data, training=True)
        loss = loss_obj(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)


@tf.function
def test_step(model, data, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(data, training=False)
    t_loss = loss_obj(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [7]:
EPOCHS = 5
"""
model_input = tf.keras.layers.Input(shape=(28, 28, 1), name="input_layer")
# x = ConvolutionalLayers([32], [3])(model_input)
# x = layers.Conv2D(32, 3, activation='relu')(model_input)
# x = layers.Flatten()(x)
x = BaseDNN1()(model_input)
# x = layers.Dense(128, activation='relu')(x)
# x = layers.Dense(10, activation='relu')(x)

model = tf.keras.Model(inputs=model_input, outputs=x)

"""

model = BaseDNN1()
# model.summary()

In [8]:
model.layers

In [9]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(model, images, labels)

  for test_images, test_labels in test_ds:
    test_step(model, test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )


Epoch 1, Loss: 2.4626150131225586, Accuracy: 10.011667251586914, Test Loss: 2.30259108543396, Test Accuracy: 10.0


KeyboardInterrupt: 